# Earth Observation Data Scientist 
We are asked to build a Python Notebook and a slide deck to assess the impact
of floodings during the Gloria storm (Januray 15th-25th, 2020) in an area of
interest of our choosing inside the Maresme county along the Tordera river
(Catalonia, Spain) with at least one of the following analytics:
1. Number of square kilometers affected by the floodings.
2. The affected population.
3. The affected roads.

The chosen AoI if the Tordera municipality.

### Imports

In [1]:
# MODULE                                      # DESCRIPTION
import sys
import matplotlib.pyplot as plt               # create visualizations
import numpy as np                            # scientific comupting
import json                                   # JSON encoder and decoder
import glob                                   # data access
import os                                     # data access
import ipywidgets                             # interactive UI controls
import time                                   # time assessment
import shutil                                 # file operations
import ipyleaflet                             # visualization
import geopandas                              # data analysis and manipulation
import snappy                                 # SNAP Python interface
import jpy                                    # Python-Java bridge
import skimage.filters                        # threshold calculation
import functools                              # higher-order functions and operations
from ipyfilechooser import FileChooser        # file chooser widget
from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt  # interface to Open Access Hub
from datetime import date                     # dates, times and intervalls
from IPython.display import display           # visualization
from osgeo import ogr, gdal, osr              # data conversion
from zipfile import ZipFile                   # file management

ImportError: DLL load failed while importing jpy: The specified module could not be found.

# Processing
### Function definition

In [ ]:
# create S1 product
def get_scene(sar_data, geojson):
    # set correct path of input file and create S1 product
    S1_source = snappy.ProductIO.readProduct(sar_data)

    # read geographic coordinates from Sentinel-1 image meta data
    meta_data = S1_source.getMetadataRoot().getElement('Abstracted_Metadata')
    # refines center of map according to Sentinel-1 image
    center = (meta_data.getAttributeDouble('centre_lat'), meta_data.getAttributeDouble('centre_lon'))
    locations = [[{'lat' : meta_data.getAttributeDouble('first_near_lat'), 'lng' : meta_data.getAttributeDouble('first_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_near_lat'),  'lng' : meta_data.getAttributeDouble('last_near_long')},
                  {'lat' : meta_data.getAttributeDouble('last_far_lat'),   'lng' : meta_data.getAttributeDouble('last_far_long')},
                  {'lat' : meta_data.getAttributeDouble('first_far_lat'),  'lng' : meta_data.getAttributeDouble('first_far_long')}]]

    # creates interactive map
    basic_map = ipyleaflet.Map(center = center, zoom = 7.5)
    # defines fixed polygon illustrating Sentinel-1 image
    polygon_fix = ipyleaflet.Polygon(locations = locations, color='royalblue')
    basic_map.add_layer(polygon_fix)
    # displays map
    basic_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
    display(basic_map)
    
    # check whether AOI file is given and convert to JSON in order to show AOI on map
    try:
        # calls readJSONfromAOI function to get GeoJSON from either JSON, SHP, or KMZ file
        data_json = readJSONFromAOI('%s/AOI' % directory)
        # show AOI on map according to JSON data
        basic_map.add_layer(ipyleaflet.GeoJSON(data = data_json, style = {'color' : 'green'}))
        # apply subset according to JSON data
        footprint = geojson_to_wkt(data_json)
        # run processing process
        processing(S1_source, footprint)
        
    # if no AOI is given, it needs to be selected manually
    except:
        # editable polygon determining AOI
        polygon_flex = ipyleaflet.Polygon(locations = locations,
                                          color = 'green',
                                          fill_color = 'green',
                                          transform = True)
        basic_map.add_layer(polygon_flex)
        # create process button and call function to start ptocessing when clicked
        processButton = ipywidgets.Button(description = 'Start Processing')
        processButton.on_click(functools.partial(on_processButton_clicked,
                                                 S1_source = S1_source,
                                                 polygon_flex = polygon_flex))
        display(processButton)
    
# calculate and return threshold of 'Band'-type input
def get_threshold(S1_band):
    # read band
    w = S1_band.getRasterWidth()
    h = S1_band.getRasterHeight()
    band_data = np.zeros(w * h, np.float32)
    S1_band.readPixels(0, 0, w, h, band_data)
    band_data.shape = h * w

    # calculate threshold using Otsu method
    threshold_otsu = skimage.filters.threshold_otsu(band_data)
    # calculate threshold using minimum method
    threshold_minimum = skimage.filters.threshold_minimum(band_data)
    # get number of pixels for both thresholds
    numPixOtsu = len(band_data[abs(band_data - threshold_otsu) < 0.1])
    numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # if number of pixels at minimum threshold is less than 1% of number of pixels at Otsu threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        # adjust band data according
        if threshold_otsu < threshold_minimum:
            band_data = band_data[band_data < threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
        else:
            band_data = band_data[band_data > threshold_minimum]
            threshold_minimum = skimage.filters.threshold_minimum(band_data)
    
        numPixMinimum = len(band_data[abs(band_data - threshold_minimum) < 0.1])

    # select final threshold
    if abs(numPixMinimum/numPixOtsu) < 0.001:
        threshold = threshold_otsu
    else:
        threshold = threshold_minimum

    return threshold

# calculate binary mask of 'Product'-type intput with respect expression in string array
def binarization(S1_product, expressions):

    BandDescriptor = jpy.get_type('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor')
    targetBands = jpy.array('org.esa.snap.core.gpf.common.BandMathsOp$BandDescriptor', len(expressions))

    # loop through bands
    for i in range(len(expressions)):
        targetBand = BandDescriptor()
        targetBand.name = '%s' % S1_product.getBandNames()[i]
        targetBand.type = 'float32'
        targetBand.expression = expressions[i]
        targetBands[i] = targetBand
    
    parameters = snappy.HashMap()
    parameters.put('targetBands', targetBands)    
    mask = snappy.GPF.createProduct('BandMaths', parameters, S1_product)

    return mask

# processing steps
def processing(S1_source, footprint):
    
    # Subset operator
    parameters = snappy.HashMap()
    parameters.put('copyMetadata', True)
    geom = snappy.WKTReader().read(footprint)
    parameters.put('geoRegion', geom)
    parameters.put('sourceBands', sourceBands)
    S1_crop = snappy.GPF.createProduct('Subset', parameters, S1_source)
    # status update
    print('\nSubset successfully generated.\n', flush=True)
    
    # Apply-Orbit-File operator
    print('1. Apply Orbit File:          ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    # continue with calculation in case no orbit file is available yet
    parameters.put('continueOnFail', True)
    S1_Orb = snappy.GPF.createProduct('Apply-Orbit-File', parameters, S1_crop)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # ThermalNoiseRemoval operator
    print('2. Thermal Noise Removal:     ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('removeThermalNoise', True)
    S1_Thm = snappy.GPF.createProduct('ThermalNoiseRemoval', parameters, S1_Orb)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Calibration operator
    print('3. Radiometric Calibration:   ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('outputSigmaBand', True)
    S1_Cal = snappy.GPF.createProduct('Calibration', parameters, S1_Thm)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Speckle-Filter operator
    print('4. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Lee')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    S1_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_Cal)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Conversion from linear to db operator
    S1_Spk_db = snappy.GPF.createProduct('LinearToFromdB', snappy.HashMap(), S1_Spk)

    # Terrain-Correction operator
    print('5. Terrain Correction:        ', end='', flush=True)
    parameters = snappy.HashMap()
    parameters.put('demName', 'SRTM 1Sec HGT')
    parameters.put('demResamplingMethod', 'BILINEAR_INTERPOLATION')
    parameters.put('imgResamplingMethod', 'NEAREST_NEIGHBOUR')
    parameters.put('pixelSpacingInMeter', 10.0)
    parameters.put('nodataValueAtSea', False)
    parameters.put('saveSelectedSourceBand', True)
    S1_TC = snappy.GPF.createProduct('Terrain-Correction', parameters, S1_Spk_db)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # Binarization
    print('6. Binarization:              ', end='', flush=True)
    start_time = time.time()
    # add GlobCover band
    parameters = snappy.HashMap()
    parameters.put('landCoverNames', 'GlobCover')
    GlobCover = snappy.GPF.createProduct('AddLandCover', parameters, S1_TC)
    # empty string array for binarization band maths expression(s)
    expressions = ['' for i in range(S1_TC.getNumBands())]
    # empty array for threshold(s)
    thresholds = np.zeros(S1_TC.getNumBands())
    # loop through bands
    for i in range(S1_TC.getNumBands()):
        # calculate threshold of band and store in float array
        # use S1_Spk_db product for performance reasons. S1_TC causes 0-values
        # which distort histogram and thus threshold result
        thresholds[i] = getThreshold(S1_Spk_db.getBandAt(i))
        # formulate expression according to threshold and store in string array
        expressions[i] = 'if (%s < %s && land_cover_GlobCover != 210) then 1 else NaN' % (S1_TC.getBandNames()[i], thresholds[i])
    # do binarization
    S1_floodMask = binarization(GlobCover, expressions)
    print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

    # Speckle-Filter operator
    print('7. Speckle Filtering:         ', end='', flush=True)
    start_time = time.time()
    parameters = snappy.HashMap()
    parameters.put('filter', 'Median')
    parameters.put('filterSizeX', 5)
    parameters.put('filterSizeY', 5)
    # define flood mask as global for later access
    global S1_floodMask_Spk
    S1_floodMask_Spk = snappy.GPF.createProduct('Speckle-Filter', parameters, S1_floodMask)
    print('--- %.2f  seconds ---' % (time.time() - start_time), flush=True)

    # output
    if plotResoluts:
        print('8. Plot:                      ', end='', flush=True)
        start_time = time.time()
        for i in range(S1_TC.getNumBands()):
            plotBand(S1_TC.getBandAt(i), thresholds[i])
        print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)


### Scene generation

In [ ]:
# filter required polarisation(s) and set output file name accordingly
source_bands = 'Amplitude_VH,Intensity_VH'
output_extensions   = 'processed_VH'

# path of Sentinel-1 .zip input file
input_path = os.path.join(directory, 'data', 'sar', 'data.zip')
footprint = os.path.join(directory, 'data', 'aoi', 'aoi.geojson')
# apply subset according to JSON data
get_scene(input_path, footprint)

# Data exporting

In [ ]:
print('Exporting...\n', flush=True)
# check if output folders exists, if not create folders
output_path = os.path.join(directory, 'output')
if not os.path.isdir(output_path):
    os.mkdir(output_path)
GeoTIFF_path = os.path.join(output_path, 'GeoTIFF')
if not os.path.isdir(GeoTIFF_path):
    os.mkdir(GeoTIFF_path)
SHP_path = os.path.join(output_path, 'SHP')
if not os.path.isdir(SHP_path):
    os.mkdir(SHP_path)
KML_path = os.path.join(output_path, 'KML')
if not os.path.isdir(KML_path):
    os.mkdir(KML_path)
GeoJSON_path = os.path.join(output_path, 'GeoJSON')
if not os.path.isdir(GeoJSON_path):
    os.mkdir(GeoJSON_path)
# get file name if file chooser was used
if len(files) is not 1: input_name = fc.selected_filename

# convert GeoTIFF to SHP
print('2. SHP:                       ', end='', flush=True)
start_time = time.time()
# allow GDAL to throw Python exceptions
gdal.UseExceptions()
open_image = gdal.Open('%s/%s_%s.tif' % (GeoTIFF_path, os.path.splitext(input_name)[0], output_extensions))
srs = osr.SpatialReference()
srs.ImportFromWkt(open_image.GetProjectionRef())
shp_driver = ogr.GetDriverByName('ESRI Shapefile')
# empty string array for bands in GeoTIFF
output_shp = ['' for i in range(open_image.RasterCount)]
if open_image.RasterCount == 1:
    output_shp[0] = '%s/%s_processed_%s' % (SHP_path, os.path.splitext(input_name)[0], polarisations)
else:
    VH_SHP_path = os.path.join(SHP_path, 'VH')
    if not os.path.isdir(VH_SHP_path):
        os.mkdir(VH_SHP_path)
    VV_SHP_path = os.path.join(SHP_path, 'VV')
    if not os.path.isdir(VV_SHP_path):
        os.mkdir(VV_SHP_path)
    output_shp[0] = '%s/%s_processed_VH' % (VH_SHP_path, os.path.splitext(input_name)[0])
    output_shp[1] = '%s/%s_processed_VV' % (VV_SHP_path, os.path.splitext(input_name)[0])
# loops through bands in GeoTIFF
for i in range(open_image.RasterCount):
    input_band = open_image.GetRasterBand(i+1)
    output_shapefile = shp_driver.CreateDataSource(output_shp[i] + '.shp')
    new_shapefile = output_shapefile.CreateLayer(output_shp[i], srs=srs)
    new_shapefile.CreateField(ogr.FieldDefn('DN', ogr.OFTInteger))
    gdal.Polygonize(input_band, input_band.GetMaskBand(), new_shapefile, 0, [], callback=None)
    # filters attributes with values other than 1 (sould be NaN or respective value)
    new_shapefile.SetAttributeFilter('DN != 1')
    for feat in new_shapefile:
        new_shapefile.DeleteFeature(feat.GetFID())
    new_shapefile.SyncToDisk()
print('--- %.2f seconds ---' % (time.time() - start_time), flush=True)

# convert SHP to GeoJSON
print('4. GeoJSON:                   ', end='', flush=True)
start_time = time.time()
if open_image.RasterCount == 1:
    shp_file = geopandas.read_file('%s/%s_processed_%s.shp' % (SHP_path, os.path.splitext(input_name)[0], polarisations))
    shp_file.to_file('%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations), driver='GeoJSON')
else:
    shp_file_VH = geopandas.read_file('%s/%s_processed_VH.shp' % (VH_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VH.to_file('%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')    
    shp_file_VV = geopandas.read_file('%s/%s_processed_VV.shp' % (VV_SHP_path, os.path.splitext(input_name)[0]))
    shp_file_VV.to_file('%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0]), driver='GeoJSON')
print('--- %.2f seconds ---\n' % (time.time() - start_time), flush=True)
print('Files successfuly stored under %s.\n' % output_path, flush=True)

# plot results
results_map = ipyleaflet.Map(zoom=9, basemap=ipyleaflet.basemaps.OpenStreetMap.Mapnik)    
if open_image.RasterCount == 1:
    file = '%s/%s_processed_%s.json' % (GeoJSON_path, os.path.splitext(input_name)[0], polarisations)
    with open(file, 'r') as f:
        data_json = json.load(f) 
    mask = ipyleaflet.GeoJSON(data = data_json, name = 'Flood Mask', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask)
    results_map.center = (mask.data['features'][0]['geometry']['coordinates'][0][0][1],
                          mask.data['features'][0]['geometry']['coordinates'][0][0][0])
else:
    file_VV = '%s/%s_processed_VV.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    with open(file_VV, 'r') as f_VV:
        data_json_VV = json.load(f_VV)
    mask_VV = ipyleaflet.GeoJSON(data = data_json_VV, name = 'Flood Mask: VV', style = {'color':'red', 'opacity':'1', 'fillColor':'red', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask_VV)
    results_map.center = (mask_VV.data['features'][0]['geometry']['coordinates'][0][0][1],
                          mask_VV.data['features'][0]['geometry']['coordinates'][0][0][0])  
    file_VH = '%s/%s_processed_VH.json' % (GeoJSON_path, os.path.splitext(input_name)[0])
    with open(file_VH, 'r') as f_VH:
        data_json_VH = json.load(f_VH)
    mask_VH = ipyleaflet.GeoJSON(data = data_json_VH, name = 'Flood Mask: VH', style = {'color':'blue', 'opacity':'1', 'fillColor':'blue', 'fillOpacity':'1', 'weight':'0.8'})
    results_map.add_layer(mask_VH)
results_map.add_control(ipyleaflet.FullScreenControl())
results_map.add_control(ipyleaflet.LayersControl(position='topright'))
results_map.add_control(ipyleaflet.ScaleControl(position='bottomleft'))
display(results_map)